In [1]:
import geopandas as gpd
import pandas as pd
import json
import os
import pygeos
import shapely
import rasterio
import json
import pyproj
import numpy as np
from pathlib import Path
from tqdm import tqdm

# custom functions
import sys
sys.path.append('../')
from utils.tiling import split_geotiff, reproject, split_annotations_to_geojson

/scistor/ivm/jpl204/miniconda3/envs/nso/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [8]:
### 1. Split geotiff args ###

raster_folder = Path("../NSO/NSO_big_tiles")
tile_output_folder = Path("../NSO/NSO_small_tiles")
#make output directory if does not exist
if not os.path.exists(tile_output_folder):
    os.makedirs(tile_output_folder)
#details of small tiles
width = 1000 #width of the new tiles
height = 1000 #height of the new tiles
nso_pix = 0.5 #pixel size of the raster

### 2. Reproject geodataframe of a annotation shapefile args ###

annotations_crs = "epsg:4326"
raster_crs = "epsg:28992"
annotation_folder = Path("../NSO/annotations_shp")
# !!!!!! Save GIS made annotations to annotation folder!!!!!! #
annotation_file = "fixed_substations_NL_annotations_06-06-23.shp"
# path to the annotation shapefile 
shp_path = os.path.join(annotation_folder, annotation_file)

### 3.Spit annotations args ###

# Make folder for geojsons
json_folder = Path("../NSO/geojsons")
if not os.path.exists(json_folder):
    os.makedirs(json_folder)

In [6]:
### Spliting a list of raster into several tiles of x width and y height skipping black tiles ###

#loop through the big tiles (rasters) and split them into smaller tiles
split_geotiff(raster_folder, tile_output_folder, width, height, nso_pix)

100%|██████████| 169/169 [3:15:33<00:00, 69.43s/it]  


In [10]:
### Reproject the DataFrame of an annotation shapefile CRS to the target CRS as the raster tiles ###

df = pd.DataFrame(gpd.read_file(shp_path).copy())

# make df geometry column as a pygeos array
df.geometry = pygeos.from_shapely(df.geometry)

# reproject geometry column
df.geometry = reproject(df, annotations_crs, raster_crs)


/scistor/ivm/jpl204/miniconda3/envs/nso/lib/python3.9/site-packages/pygeos/io.py:89: UserWarning: The shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the PyGEOS GEOS version (3.10.4-CAPI-1.16.2). Conversions between both will be slow
  warnings.warn(


In [ ]:
### Split annotations DataFrame into the new small tiles of width X and height Y & save it to geojson ###

split_annotations_to_geojson(df, tile_output_folder, json_folder)